In [1]:
# import the library
import pandas as pd

In [2]:
# loading the data from the excel file
cleaner_trade = pd.read_excel("C:\\Users\\rithi\\Downloads\\clearer_trade_data.xlsx", sheet_name = "Sheet1")
internal_trade_data = pd.read_excel("C:\\Users\\rithi\\Downloads\\internal_trade_data.xlsx", sheet_name = "Sheet1")

In [3]:
# changing the internal price data to float
internal_trade_data.price_eur_per_mwh = internal_trade_data.price_eur_per_mwh.astype(float)

Dividing the Quaterly contract equally across three 


In [4]:
Q225_break = float(internal_trade_data.quantity_mwh[3] / 3)

In [5]:
#Computing Outrightaddition
outrightaddition = internal_trade_data.copy()
outrightaddition.quantity_mwh[1]+=Q225_break
outrightaddition.quantity_mwh[2]+=Q225_break
outrightaddition

C:\Users\rithi\AppData\Local\Temp\ipykernel_22112\792803375.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  outrightaddition.quantity_mwh[1]+=Q225_break
C:\Users\rithi\AppData\Local\Temp\ipykernel_22112\792803375.py:3: SettingWithCopyWar

,trade_id,contract_type,contract_period,quantity_mwh,price_eur_per_mwh
0,1,Outright,2025-03,100,30.0
1,2,Outright,2025-04,300,31.0
2,3,Outright,2025-05,350,32.0
3,4,Outright Quarter,Q225,300,33.0


In [6]:
# Weighted average of the price
outrightprice1 = ((internal_trade_data.quantity_mwh[1] * internal_trade_data.price_eur_per_mwh[1])+ (Q225_break*internal_trade_data.price_eur_per_mwh[3]))/(internal_trade_data.quantity_mwh[1] + Q225_break)
outrightprice2 = ((internal_trade_data.quantity_mwh[2] * internal_trade_data.price_eur_per_mwh[2])+ (Q225_break*internal_trade_data.price_eur_per_mwh[3]))/(internal_trade_data.quantity_mwh[2] + Q225_break)

changing the price per month to the calculated price 

In [7]:
# Upading the price variables
outrightaddition.price_eur_per_mwh[1] = outrightprice1
outrightaddition.price_eur_per_mwh[2] = outrightprice2
outrightaddition

C:\Users\rithi\AppData\Local\Temp\ipykernel_22112\1876285025.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  outrightaddition.price_eur_per_mwh[1] = outrightprice1
C:\Users\rithi\AppData\Local\Temp\ipykernel_22112\1876285025.py:2: Settin

,trade_id,contract_type,contract_period,quantity_mwh,price_eur_per_mwh
0,1,Outright,2025-03,100,30.000000
1,2,Outright,2025-04,300,31.666667
2,3,Outright,2025-05,350,32.285714
3,4,Outright Quarter,Q225,300,33.000000


In [8]:
#Removing the outright quater for further calculation
outrightaddition1 = outrightaddition.drop(3)

Finding the discrepency

In [9]:
#step 1 - Adding the month of June into the internal dataset.
new_row = pd.DataFrame([{
    'trade_id' : 5,
    'contract_type': 'Outright',
    'contract_period': '2025-06',
    'quantity_mwh': 100,
    'price_eur_per_mwh': 33.0
}])
updatedoutright = pd.concat([outrightaddition1, new_row], ignore_index=True) 

In [10]:
# Step 2 - Calculation of Discrepency
for i in range(4):
    for j in range(4):
        if updatedoutright.contract_period[i] == cleaner_trade.delivery_month[j]:
            print(f"In the month {updatedoutright.contract_period[i]}, the discrepancy in quantity is {updatedoutright.quantity_mwh[i]-cleaner_trade.quantity_mwh[j]}MWh and the price is {updatedoutright.price_eur_per_mwh[i]-cleaner_trade.price_eur_per_mwh[j]} Euros")

In the month 2025-03, the discrepancy in quantity is 0MWh and the price is 0.0 Euros
In the month 2025-04, the discrepancy in quantity is -200MWh and the price is -0.33333333333333215 Euros
In the month 2025-05, the discrepancy in quantity is -200MWh and the price is -0.2142857142857153 Euros
In the month 2025-06, the discrepancy in quantity is -200MWh and the price is 0.0 Euros
